In [ ]:
#!pip install -r '/Users/akhilgolla/Documents/Careery/ML Projects/Q&A Tool Retail Domain/requirements.txt'

In [ ]:
#pip install --upgrade langchain pydantic sqlalchemy pymysql

In [1]:
import sys
sys.path.append('/Users/akhilgolla/Documents/Careery/ML Projects/Hugging Face Project/Langchain Codebasics')

from Secret_Key import openapi_key

In [2]:
import os
os.environ['OPENAI_API_KEY'] = openapi_key

In [3]:
#openapi_key

In [4]:
from typing import Any

# Placeholder for the missing `Callbacks` annotation
Callbacks = Any

In [5]:
from langchain.llms import OpenAI

#Using OpenAI llm here but can use any llm like huggingface, llama2, mistral etc.

llm = OpenAI(temperature=0.2)
#poem = llm("Write a poem about biryani.")

/var/folders/5k/ty4rqtyn711100q4g5b3tc7w0000gn/T/ipykernel_6165/1441013698.py:5: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(temperature=0.2)


In [6]:
#print(poem)

In [7]:
from langchain.utilities import SQLDatabase

In [8]:
from urllib.parse import quote

db_user = "root"
db_password = quote("Highness@6515")  # URL encode the password
db_host = "localhost"
db_port = 3306
db_name = "atliq_tshirts"

# Create the database connection with encoded password
db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}", sample_rows_in_table_info=3)

# Print table information to verify the connection
print(db.table_info)


CREATE TABLE discounts (
	discount_id INTEGER NOT NULL AUTO_INCREMENT, 
	t_shirt_id INTEGER NOT NULL, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (discount_id), 
	CONSTRAINT discounts_ibfk_1 FOREIGN KEY(t_shirt_id) REFERENCES t_shirts (t_shirt_id), 
	CONSTRAINT discounts_chk_1 CHECK ((`pct_discount` between 0 and 100))
)ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci DEFAULT CHARSET=utf8mb4

/*
3 rows from discounts table:
discount_id	t_shirt_id	pct_discount
1	1	10.00
2	2	15.00
3	3	20.00
*/


CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci DEFAULT CHARSET=utf8mb4

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock

In [9]:
#from langchain import SQLDatabaseChain
from langchain_experimental.sql import SQLDatabaseChain
from langchain.schema import BaseCache

In [10]:
pip show pydantic

Name: pydantic
Version: 2.10.4
Summary: Data validation using Python type hints
Home-page: 
Author: 
Author-email: Samuel Colvin <s@muelcolvin.com>, Eric Jolibois <em.jolibois@gmail.com>, Hasan Ramezani <hasan.r67@gmail.com>, Adrian Garcia Badaracco <1755071+adriangb@users.noreply.github.com>, Terrence Dorsey <terry@pydantic.dev>, David Montague <david@pydantic.dev>, Serge Matveenko <lig@countzero.co>, Marcelo Trylesinski <marcelotryle@gmail.com>, Sydney Runkle <sydneymarierunkle@gmail.com>, David Hewitt <mail@davidhewitt.io>, Alex Hall <alex.mojaki@gmail.com>, Victorien Plot <contact@vctrn.dev>
License: 
Location: /Users/akhilgolla/anaconda3/lib/python3.11/site-packages
Requires: annotated-types, pydantic-core, typing-extensions
Required-by: anaconda-cloud-auth, chromadb, fastapi, google-generativeai, langchain, langchain-core, langsmith, openai, pydantic-settings
Note: you may need to restart the kernel to use updated packages.


In [11]:
SQLDatabaseChain.model_rebuild()

True

In [12]:
#pip install --upgrade langchain

In [13]:
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

In [23]:
query = "How many t-shirts do we have left for Nike in extra small size and white color?"
qns1 = db_chain(query)
print(qns1)



> Entering new SQLDatabaseChain chain...
How many t-shirts do we have left for Nike in extra small size and white color?
SQLQuery:SELECT SUM(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND size = 'XS' AND color = 'White'
SQLResult: [(Decimal('74'),)]
Answer:74
> Finished chain.
{'query': 'How many t-shirts do we have left for Nike in extra small size and white color?', 'result': '74'}


In [15]:
qns2 = db_chain.run("How much is the price of the inventory for all small size t-shirts?")

/var/folders/5k/ty4rqtyn711100q4g5b3tc7w0000gn/T/ipykernel_6165/521761031.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  qns2 = db_chain.run("How much is the price of the inventory for all small size t-shirts?")




> Entering new SQLDatabaseChain chain...
How much is the price of the inventory for all small size t-shirts?
SQLQuery:SELECT SUM(price * stock_quantity) AS total_price FROM t_shirts WHERE size = 'S'
SQLResult: [(Decimal('15590'),)]
Answer:The total price of the inventory for all small size t-shirts is $155.90.
> Finished chain.


In [16]:
#This will give wrong output or error
qns3 = db_chain.run("If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?")



> Entering new SQLDatabaseChain chain...
If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?
SQLQuery:SELECT SUM(discounts.pct_discount * t_shirts.price / 100) AS revenue
FROM discounts
INNER JOIN t_shirts ON discounts.t_shirt_id = t_shirts.t_shirt_id
WHERE t_shirts.brand = 'Levi' AND CURDATE() = 'today'
LIMIT 5;

OperationalError: (pymysql.err.OperationalError) (1525, "Incorrect DATE value: 'today'")
[SQL: SELECT SUM(discounts.pct_discount * t_shirts.price / 100) AS revenue
FROM discounts
INNER JOIN t_shirts ON discounts.t_shirt_id = t_shirts.t_shirt_id
WHERE t_shirts.brand = 'Levi' AND CURDATE() = 'today'
LIMIT 5;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [17]:
sql_code = """
select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """

qns3 = db_chain.run(sql_code)



> Entering new SQLDatabaseChain chain...

select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 
SQLQuery:SELECT SUM(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) AS total_revenue 
FROM (SELECT SUM(price*stock_quantity) AS total_amount, t_shirt_id 
FROM t_shirts 
WHERE brand = 'Levi' 
GROUP BY t_shirt_id) a 
LEFT JOIN discounts ON a.t_shirt_id = discounts.t_shirt_id
SQLResult: [(Decimal('19987.700000'),)]
Answer:The total revenue for Levi brand t-shirts after applying discounts is $19,987.70.
> Finished chain.


In [18]:
qns3

'The total revenue for Levi brand t-shirts after applying discounts is $19,987.70.'

In [19]:
qns4 = db_chain.run("SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'")



> Entering new SQLDatabaseChain chain...
SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLQuery:SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLResult: [(Decimal('21329'),)]
Answer:The total revenue for all Levi t-shirts in stock is $21,329.
> Finished chain.


In [20]:
#This will give wrong answer again
qns5 = db_chain.run("How many white color Levi's t shirts we have available?")



> Entering new SQLDatabaseChain chain...
How many white color Levi's t shirts we have available?
SQLQuery:SELECT COUNT(*) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(5,)]
Answer:5
> Finished chain.


In [21]:
qns5 = db_chain.run("SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'")



> Entering new SQLDatabaseChain chain...
SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLQuery:SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(Decimal('360'),)]
Answer:360
> Finished chain.


In [35]:
few_shots = [
    {'Question' : "How many t-shirts do we have left for Nike in XS size and white color?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
     'SQLResult': "Result of the SQL query",
     'Answer' : qns1},
    {'Question': "How much is the total price of the inventory for all S-size t-shirts?",
     'SQLQuery':"SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'",
     'SQLResult': "Result of the SQL query",
     'Answer': qns2},
    {'Question': "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?" ,
     'SQLQuery' : """SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """,
     'SQLResult': "Result of the SQL query",
     'Answer': qns3} ,
     {'Question' : "If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?" ,
      'SQLQuery': "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'",
      'SQLResult': "Result of the SQL query",
      'Answer' : qns4},
    {'Question': "How many white color Levi's shirt I have?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
     'SQLResult': "Result of the SQL query",
     'Answer' : qns5
     }
]

In [36]:
#Creating Semantic Similarity Based example selector
#create embedding on the few_shots
#Store the embeddings in Chroma DB
#Retrieve the the top most Semantically close example from the vector store

In [42]:
from langchain.prompts import SemanticSimilarityExampleSelector
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma


embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

# Corrected Code
#to_vectorize = [" ".join([str(value) if not isinstance(value, str) else value for value in example.values()]) for example in few_shots]
to_vectorize = [" ".join(map(str, example.values())) for example in few_shots]

print(to_vectorize)  # This will display the processed text ready for vectorization

["How many t-shirts do we have left for Nike in XS size and white color? SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS' Result of the SQL query {'query': 'How many t-shirts do we have left for Nike in extra small size and white color?', 'result': '74'}", "How much is the total price of the inventory for all S-size t-shirts? SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S' Result of the SQL query The total price of the inventory for all small size t-shirts is $155.90.", "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)? SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from\n(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'\ngroup by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id\n  Result of the SQL query The total revenue fo

In [45]:
to_vectorize[0]

"How many t-shirts do we have left for Nike in XS size and white color? SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS' Result of the SQL query {'query': 'How many t-shirts do we have left for Nike in extra small size and white color?', 'result': '74'}"

In [51]:
# Serialize metadata values to strings
flattened_metadata = [
    {key: (str(value) if isinstance(value, (dict, list)) else value) for key, value in example.items()}
    for example in few_shots
]

In [52]:
# Use flattened metadata
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=flattened_metadata)

In [53]:
vectorstore

In [54]:
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2,
)

example_selector.select_examples({"Question": "How many Adidas T shirts I have left in my store?"})

[{'Answer': "{'query': 'How many t-shirts do we have left for Nike in extra small size and white color?', 'result': '74'}",
  'Question': 'How many t-shirts do we have left for Nike in XS size and white color?',
  'SQLQuery': "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
  'SQLResult': 'Result of the SQL query'},
 {'Answer': "{'query': 'How many t-shirts do we have left for Nike in extra small size and white color?', 'result': '74'}",
  'Question': 'How many t-shirts do we have left for Nike in XS size and white color?',
  'SQLQuery': "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
  'SQLResult': 'Result of the SQL query'}]

In [60]:
### my sql based instruction prompt
mysql_prompt = """You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: Query to run with no pre-amble
SQLResult: Result of the SQLQuery
Answer: Final answer here

No pre-amble.
"""

In [61]:
from langchain.prompts import FewShotPromptTemplate
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _mysql_prompt

print(PROMPT_SUFFIX)

Only use the following tables:
{table_info}

Question: {input}


In [ ]:
#Setting up PromptTemplete using input variables

In [62]:
from langchain.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate(
    input_variables=["Question", "SQLQuery", "SQLResult","Answer",],
    template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}",
)

In [63]:

print(_mysql_prompt)

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of

In [64]:
few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=mysql_prompt,
    suffix=PROMPT_SUFFIX,
    input_variables=["input", "table_info", "top_k"], #These variables are used in the prefix and suffix
)

In [65]:
new_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, prompt=few_shot_prompt)

In [66]:
new_chain("How many white color Levi's shirt I have?")



> Entering new SQLDatabaseChain chain...
How many white color Levi's shirt I have?
SQLQuery:SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(Decimal('360'),)]
Answer:360
> Finished chain.


{'query': "How many white color Levi's shirt I have?", 'result': '360'}

In [67]:
new_chain("How much is the price of the inventory for all small size t-shirts?")



> Entering new SQLDatabaseChain chain...
How much is the price of the inventory for all small size t-shirts?
SQLQuery:SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'
SQLResult: [(Decimal('15590'),)]
Answer:The price of the inventory for all small size t-shirts is $15,590.
> Finished chain.


{'query': 'How much is the price of the inventory for all small size t-shirts?',
 'result': 'The price of the inventory for all small size t-shirts is $15,590.'}

In [68]:
new_chain("How much is the price of all white color levi t shirts?")



> Entering new SQLDatabaseChain chain...
How much is the price of all white color levi t shirts?
SQLQuery:SELECT sum(price) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(Decimal('169'),)]
Answer:169
> Finished chain.


{'query': 'How much is the price of all white color levi t shirts?',
 'result': '169'}

In [69]:
new_chain("If we have to sell all the Nike’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?")



> Entering new SQLDatabaseChain chain...
If we have to sell all the Nike’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?
SQLQuery:SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Nike'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
SQLResult: [(Decimal('17918.600000'),)]
Answer:The total revenue for Nike brand t-shirts after applying discounts is $17,918.60.
> Finished chain.


{'query': 'If we have to sell all the Nike’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?',
 'result': 'The total revenue for Nike brand t-shirts after applying discounts is $17,918.60.'}

In [70]:
new_chain("If we have to sell all the Van Heuson T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?")



> Entering new SQLDatabaseChain chain...
If we have to sell all the Van Heuson T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?
SQLQuery:SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Van Huesen'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
SQLResult: [(Decimal('33417.200000'),)]
Answer:The total revenue for Van Heusen brand t-shirts after applying discounts is $33,417.20.
> Finished chain.


{'query': 'If we have to sell all the Van Heuson T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?',
 'result': 'The total revenue for Van Heusen brand t-shirts after applying discounts is $33,417.20.'}

In [71]:
new_chain.run('How much revenue  our store will generate by selling all Van Heuson TShirts without discount?')



> Entering new SQLDatabaseChain chain...
How much revenue  our store will generate by selling all Van Heuson TShirts without discount?
SQLQuery:SELECT sum(price*stock_quantity) as total_revenue from t_shirts where brand = 'Van Huesen'
SQLResult: [(Decimal('34185'),)]
Answer:The total revenue for Van Heusen brand t-shirts without any discounts is $34,185.
> Finished chain.


'The total revenue for Van Heusen brand t-shirts without any discounts is $34,185.'